# **Chat with Bengali Data 01:**

## **Intall Libaries:**

In [ ]:
!pip -q install langchain
!pip -q install huggingface-hub
!pip -q install sentence-transformers==2.2.2
!pip -q install pypdf
!pip -q install PyPDF2
!pip -q install pinecone-client

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 817.7/817.7 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 13.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 291.3/291.3 kB 11.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.2/115.2 kB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.1/141.1 kB 12.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 2.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.4/290.4 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 215.9/215.9 kB 4.5 MB/s eta 0:00:00


## **Import Modules:**

In [ ]:
import os
import PyPDF2
import pinecone
from langchain import HuggingFaceHub
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Pinecone
# from pinecone import Pinecone, ServerlessSpec
from langchain.prompts import PromptTemplate, ChatPromptTemplate
from langchain.chains import LLMChain, SequentialChain, ConversationChain, RetrievalQA, QAGenerationChain, ConversationalRetrievalChain
from langchain.chains.question_answering import load_qa_chain
from langchain.memory import ConversationBufferWindowMemory, ConversationBufferMemory
from google.colab import userdata
from IPython.display import display, Markdown

In [ ]:
ROOT = "/content/drive/MyDrive/Generative AI/End-to-End Projects/2. K12-Chatbot"
os.chdir(ROOT)

!pwd

/content/drive/MyDrive/Generative AI/End-to-End Projects/2. K12-Chatbot


## **Load Data:**

### **Merge Pdfs:**

In [ ]:
# Get a list of all PDF files in the directory
pdf_directory = "Data"
pdf_files = [f for f in os.listdir(pdf_directory) if f.endswith(".pdf")]
# print(pdf_files)

# Create a PDF merger object
pdf_merger = PyPDF2.PdfWriter()


# Loop through the list of PDF files and merge them
for pdf_file in sorted(pdf_files): # Sorting to maintain a consistent order
  pdf_path = os.path.join(pdf_directory, pdf_file)  # Get the full path
  # print(pdf_path)
  with open(pdf_path, "rb") as file:
        pdf_reader = PyPDF2.PdfReader(file)
        # Add all pages from the current PDF to the merger
        for page in range(len(pdf_reader.pages)):
            pdf_merger.add_page(pdf_reader.pages[page])


# Write the merged content to a new PDF file
merge_pdf_dir = "Data/ben_pdfs.pdf"
with open(merge_pdf_dir, "wb") as output_file:
    pdf_merger.write(output_file)

### **Load as Documents:**

In [ ]:
# create a function for loading the data and get PDF documents:

main_pdf_file_path = "Data/ben_pdfs.pdf"
def load_data(file_path=main_pdf_file_path):
  loader = PyPDFLoader(file_path=file_path)
  documents = loader.load()
  return documents

docs = load_data()
docs

In [ ]:
len(docs)

63

### **Chunkings:**

In [ ]:
# create chunks:

def create_chunks(documents):
  text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=200)
  texts_chunks = text_splitter.split_documents(documents)
  return texts_chunks

texts_chunks = create_chunks(documents=docs)
texts_chunks

In [ ]:
len(texts_chunks)

60

## **Load Embeddings Model:**

In [ ]:
HF_TOKEN = userdata.get('HF_TOKEN')
os.environ["HF_TOKEN"] = HF_TOKEN

In [ ]:
# Create Embeddings:

embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

.gitattributes:   0%|          | 0.00/1.23k [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

train_script.py:   0%|          | 0.00/13.2k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

## **Create Knowledge Base:**

In [ ]:
# get the pinecone API key:

PINECONE_API_KEY = userdata.get('PINECONE_API')
os.environ["PINECONE_API_KEY"] = PINECONE_API_KEY
PINECONE_API_ENV = "us-east-1"

### **Store Embedding in Pinecone:**

In [ ]:
# create & insert Embeddings into Pinecone:

from langchain.vectorstores import Pinecone
docsearch = Pinecone.from_texts([t.page_content for t in texts_chunks], embeddings, index_name="mychatpdf")

In [ ]:
# If you already have an index, you can load it like this:

docsearch = Pinecone.from_existing_index(index_name="mychatpdf", embedding=embeddings)
docsearch

In [ ]:
# Retrieve Information:

query = "বলের দ্বারা কার্য কাকে বলে?"
docs = docsearch.similarity_search(query, k=30) # retrive top 100 relevent or similar documents
docs

## **Load LLM:**

In [ ]:
!pip -q install langchain_google_genai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 146.8/146.8 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.5/664.5 kB 16.4 MB/s eta 0:00:00


In [ ]:
# Get the Token:

HUGGINGFACEHUB_API_TOKEN = userdata.get('HF_TOKEN')
GEMINI_API_KEY = userdata.get('GEMINI_API_KEY') # get the Gemini API Key

In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI

llm_mistralai = HuggingFaceHub(repo_id="mistralai/Mistral-7B-Instruct-v0.2", huggingfacehub_api_token=HUGGINGFACEHUB_API_TOKEN,
                                   model_kwargs={"temperature":0.4, "max_length":512})

llm_gemini = ChatGoogleGenerativeAI(model="gemini-pro", google_api_key=GEMINI_API_KEY, temperature=0, max_length=512)

In [ ]:
llm_gemini

ChatGoogleGenerativeAI(model='gemini-pro', google_api_key=SecretStr('**********'), temperature=0.0, client=genai.GenerativeModel(
    model_name='models/gemini-pro',
    generation_config={},
    safety_settings={},
    tools=None,
    system_instruction=None,
))

## **Generate Response:**

### **Method 1:**

In [ ]:

qa_template = """
Use the following information from the context (separated with ) to answer the question in `Bengali` language.
Please remember that user don't know other languages except `Bengali`.
Remember when user ask `question` out of context then simply answer "Unfortunately, I don't have the information" in just oneline.
If you don't know the answer, answer with "Unfortunately, I don't have the information." \
If you don't find enough information below, also answer with "Unfortunately, I don't have enough information." \
------

{context}

------

{chat_history}

------
{question}
Helpful Answer:
"""
qa_prompt = PromptTemplate(template=qa_template, input_variables=['context', 'chat_history', 'question'])

custom_summary_prompt='''
Generate the overall summary of the following text (within the 512 words) that includes the following below elements:

* A title that accurately reflects the content of the text.
* An introduction paragraph that provides an overview of the topic.
* Bullet points that list the key points of the text.
* A conclusion paragraph that summarizes the main points of the text.

Text:`{context}`
'''
custom_summary_template = PromptTemplate(template=custom_summary_prompt, input_variables=['context'])


# define memory:
memory = ConversationBufferWindowMemory(
    llm=llm_gemini,
    memory_key="chat_history",
    input_key="question",
    k=5
)


chain_type_kwargs={
        # "verbose": True,
        "question_prompt": qa_prompt,
        "combine_prompt": custom_summary_template,
        "combine_document_variable_name": "context",
        "memory": memory
}

qa_chain = RetrievalQA.from_chain_type(llm=llm_gemini,
                                 chain_type="map_reduce",
                                #  return_source_documents=True,
                                 chain_type_kwargs=chain_type_kwargs,
                                 retriever=docsearch.as_retriever(search_kwargs={"k": 15})
                                 )

In [ ]:
memory

ConversationBufferWindowMemory(input_key='question', memory_key='chat_history')

In [ ]:
# Markdown Function:

def generate_response(question, qa_chain=qa_chain):
  ans = qa_chain.invoke(question)
  return display(Markdown(ans['result']))

In [ ]:
generate_response(question="বলের দ্বারা কার্য কাকে বলে?")

**বল এবং কার্য: একটি সংক্ষিপ্ত বিশ্লেষণ**

**পরিচিতি**

পদার্থবিজ্ঞানে, বল এবং কার্য দুটি মৌলিক ধারণা যা বস্তুর গতি এবং শক্তির সাথে সম্পর্কিত। এই নিবন্ধটি বল এবং কার্যের সংজ্ঞা, তাদের মধ্যে সম্পর্ক এবং পদার্থবিজ্ঞানে তাদের প্রাসঙ্গিকতা অন্বেষণ করে।

**বল এবং কার্যের সংজ্ঞা**

* **বল:** বল হলো একটি ভেক্টর রাশি যা একটি বস্তুর উপর প্রযুক্ত হয় এবং তার গতি বা আকার পরিবর্তন করতে পারে।
* **কার্য:** কার্য হলো একটি স্কেলার রাশি যা একটি বল দ্বারা একটি বস্তুর সরণের দিকে প্রযুক্ত বলের গুণফল দ্বারা নির্ধারিত হয়।

**বল এবং কার্যের মধ্যে সম্পর্ক**

বল এবং কার্যের মধ্যে একটি সরাসরি সম্পর্ক রয়েছে। একটি বস্তুর উপর প্রযুক্ত বল যত বেশি হবে, তত বেশি কার্য সম্পাদিত হবে। তদুপরি, বলের দিক এবং বস্তুর সরণের দিকের মধ্যে কোণ যত কম হবে, তত বেশি কার্য সম্পাদিত হবে।

**পদার্থবিজ্ঞানে বল এবং কার্যের প্রাসঙ্গিকতা**

বল এবং কার্য পদার্থবিজ্ঞানের বিভিন্ন ক্ষেত্রে গুরুত্বপূর্ণ ভূমিকা পালন করে, যেমন:

* **যান্ত্রিকতা:** বল এবং কার্য যন্ত্রের কার্যকারিতা ব্যাখ্যা করতে ব্যবহৃত হয়, যেমন লিভার, পুলি এবং র্যাম্প।
* **গতিবিদ্যা:** বল এবং কার্য বস্তুর গতি এবং ত্বরণ ব্যাখ্যা করতে ব্যবহৃত হয়।
* **শক্তি:** কার্য শক্তির একটি রূপ হিসাবে বিবেচিত হয়, এবং এটি বস্তুর শক্তির পরিবর্তন পরিমাপ করতে ব্যবহৃত হয়।

**উদাহরণ**

বল এবং কার্যের একটি সাধারণ উদাহরণ হলো একটি বইকে টেবিলের উপর তোলা। বইয়ের উপর প্রযুক্ত বল হলো মাধ্যাকর্ষণ বল, এবং বইয়ের সরণ হলো টেবিলের উপরের দিকে তার উচ্চতা। মাধ্যাকর্ষণ বল এবং বইয়ের সরণের দিকের মধ্যে কোণ 180 ডিগ্রি, যার অর্থ কার্য নেতিবাচক।

**উপসংহার**

বল এবং কার্য পদার্থবিজ্ঞানে দুটি মৌলিক ধারণা যা বস্তুর গতি এবং শক্তির সাথে সম্পর্কিত। বল একটি বস্তুর উপর প্রযুক্ত হয় এবং তার গতি বা আকার পরিবর্তন করতে পারে, যখন কার্য একটি বল দ্বারা একটি বস্তুর সরণের দিকে প্রযুক্ত বলের গুণফল দ্বারা নির্ধারিত হয়। বল এবং কার্যের মধ্যে একটি সরাসরি সম্পর্ক রয়েছে, এবং এগুলি পদার্থবিজ্ঞানের বিভিন্ন ক্ষেত্রে গুরুত্বপূর্ণ ভূমিকা পালন করে।

In [ ]:
generate_response(question="তোমার সম্পর্কে আমাকে কিছু বলো?")

In [ ]:
generate_response(question="Who you are?")

**গুগল কর্তৃক প্রশিক্ষিত বৃহৎ ভাষা মডেল: একটি পরিচিতি**

**পরিচয়**

বৃহৎ ভাষা মডেল (LLM) কৃত্রিম বুদ্ধিমত্তা (AI) ভিত্তিক সিস্টেম যা মানুষের ভাষা বোঝার এবং উৎপাদন করার জন্য প্রশিক্ষিত। গুগল কর্তৃক প্রশিক্ষিত একটি LLM হিসাবে, এই পাঠ্যটি আমার ক্ষমতা, সীমাবদ্ধতা এবং ভাষা-সম্পর্কিত কাজে সহায়তা করার আমার ভূমিকা সম্পর্কে একটি সংক্ষিপ্ত বিবরণ প্রদান করে।

**মূল বিন্দু**

* আমি একটি LLM, যার অর্থ আমি বিশাল পরিমাণ পাঠ্য ডেটাসেটে প্রশিক্ষিত হয়েছি।
* গুগল আমাকে প্রশিক্ষণ দিয়েছে, যার অর্থ আমি তাদের অত্যাধুনিক AI প্রযুক্তি দ্বারা সমর্থিত।
* আমার কোনো নাম নেই কারণ আমি একটি কম্পিউটার প্রোগ্রাম, কোনো ব্যক্তি নই।
* আমি অনুবাদ, প্রশ্নোত্তর এবং পাঠ্য উৎপাদন সহ বিভিন্ন ভাষা-সম্পর্কিত কাজে দক্ষ।
* আমার উদ্দেশ্য মানুষকে তাদের ভাষা-সম্পর্কিত কাজে সহায়তা করা, যেমন তথ্য অনুসন্ধান, যোগাযোগ এবং সৃজনশীল লেখা।

**উপসংহার**

গুগল কর্তৃক প্রশিক্ষিত একটি LLM হিসাবে, আমি ভাষা-সম্পর্কিত কাজে মানুষকে সহায়তা করার জন্য ডিজাইন করা হয়েছে। আমার ক্ষমতা এবং সীমাবদ্ধতা বোঝার মাধ্যমে, ব্যবহারকারীরা আমাকে তাদের কাজের দক্ষতা এবং কার্যকারিতা উন্নত করতে কাজে লাগাতে পারেন। আমি ভাষা প্রযুক্তির ক্ষেত্রে অবিরাম অগ্রগতির প্রতিনিধিত্ব করি এবং ভবিষ্যতে মানুষের সাথে যোগাযোগ এবং তথ্য অ্যাক্সেস করার উপায়কে রূপান্তরিত করার সম্ভাবনা রাখি।

In [ ]:
generate_response(question="Tell me something about Force?")


**বল: গতি পরিবর্তনকারী একটি ভেক্টর রাশি**

**পরিচয়**

বল একটি ভৌত রাশি যা বস্তুর গতি পরিবর্তন করতে পারে। এটি একটি ভেক্টর রাশি যার মান এবং দিক উভয়ই রয়েছে। বল আমাদের চারপাশের বিশ্বকে বোঝার জন্য একটি মৌলিক ধারণা।

**বলের বৈশিষ্ট্য**

* ভেক্টর রাশি: বলের একটি মান এবং একটি দিক রয়েছে।
* মাত্রা: বল নিউটন (N) এককে পরিমাপ করা হয়।
* দিক: বলের দিক নির্দেশ করে যে দিকে এটি প্রয়োগ করা হচ্ছে।

**বলের প্রকারভেদ**

* **স্পর্শ বল:** দুটি বস্তুর সরাসরি সংস্পর্শে থাকার সময় প্রয়োগ করা হয়।
* **দূরত্ব বল:** দুটি বস্তুর মধ্যে দূরত্ব থাকা সত্ত্বেও প্রয়োগ করা হয়।
* **চুম্বকীয় বল:** চুম্বকীয় ক্ষেত্রের মধ্যে চার্জযুক্ত কণা বা চুম্বকীয় উপকরণের উপর প্রয়োগ করা হয়।

**বলের প্রভাব**

* **গতি পরিবর্তন:** বল একটি বস্তুর গতি, দিক বা উভয়ই পরিবর্তন করতে পারে।
* **আকার পরিবর্তন:** বল একটি বস্তুর আকার বা আকৃতি পরিবর্তন করতে পারে।
* **বস্তুর অবস্থা পরিবর্তন:** কিছু ক্ষেত্রে, বল একটি বস্তুর অবস্থা পরিবর্তন করতে পারে।

**বলের গাণিতিক প্রকাশ**

নিউটনের দ্বিতীয় গতিসূত্র অনুসারে, একটি বস্তুর উপর প্রয়োগ করা বল (F) বস্তুর ভর (m) এবং ত্বরণ (a) এর গুণফলের সমান:

F = m * a

**উদাহরণ**

বলের কিছু সাধারণ উদাহরণ হল:

* একটি বইকে টেবিলের উপর ধাক্কা দেওয়া
* একটি বলকে লাথি মারা
* একটি চুম্বক একটি পেপারক্লিপকে আকর্ষণ করা
* পৃথিবীর মহাকর্ষ বল যা আমাদেরকে মাটিতে রাখে

**উপসংহার**

বল একটি মৌলিক ভৌত রাশি যা আমাদের চারপাশের বিশ্বকে বোঝার জন্য অপরিহার্য। এটি বস্তুর গতি, আকার এবং অবস্থা পরিবর্তন করতে পারে এবং আমাদের দৈনন্দিন জীবনের বিভিন্ন ক্ষেত্রে প্রয়োগ করা হয়।

In [ ]:
memory

ConversationBufferWindowMemory(chat_memory=ChatMessageHistory(messages=[HumanMessage(content='বলের দ্বারা কার্য কাকে বলে?'), AIMessage(content='**বল এবং কার্য: একটি সংক্ষিপ্ত বিশ্লেষণ**\n\n**পরিচিতি**\n\nপদার্থবিজ্ঞানে, বল এবং কার্য দুটি মৌলিক ধারণা যা বস্তুর গতি এবং শক্তির সাথে সম্পর্কিত। এই নিবন্ধটি বল এবং কার্যের সংজ্ঞা, তাদের মধ্যে সম্পর্ক এবং পদার্থবিজ্ঞানে তাদের প্রাসঙ্গিকতা অন্বেষণ করে।\n\n**বল এবং কার্যের সংজ্ঞা**\n\n* **বল:** বল হলো একটি ভেক্টর রাশি যা একটি বস্তুর উপর প্রযুক্ত হয় এবং তার গতি বা আকার পরিবর্তন করতে পারে।\n* **কার্য:** কার্য হলো একটি স্কেলার রাশি যা একটি বল দ্বারা একটি বস্তুর সরণের দিকে প্রযুক্ত বলের গুণফল দ্বারা নির্ধারিত হয়।\n\n**বল এবং কার্যের মধ্যে সম্পর্ক**\n\nবল এবং কার্যের মধ্যে একটি সরাসরি সম্পর্ক রয়েছে। একটি বস্তুর উপর প্রযুক্ত বল যত বেশি হবে, তত বেশি কার্য সম্পাদিত হবে। তদুপরি, বলের দিক এবং বস্তুর সরণের দিকের মধ্যে কোণ যত কম হবে, তত বেশি কার্য সম্পাদিত হবে।\n\n**পদার্থবিজ্ঞানে বল এবং কার্যের প্রাসঙ্গিকতা**\n\nবল এবং কার্য পদার্থবিজ্ঞানের বিভিন্ন ক্ষে

### **Method 2:**

In [ ]:
qa_template = """
You are an AI-powered virtual assistant, named Mini, designed to help K-12 students with their studies.
The student is in grade {grade}, studying {subject}.
They are interested in the topic '{topic}'.
Their question is: {question}.
Please provide a clear and concise response that is appropriate for a K-12 student.

Use the following information from the context (separated with ) to answer the question in `Bengali` language.
Remember that the user doesn't know other languages except `Bengali`.
If the question is out of context, answer "Unfortunately, I don't have the information." in just one line.
If you don't know the answer, say "Unfortunately, I don't have the information."
If you don't find enough information below, answer "Unfortunately, I don't have enough information."
------

{context}

------
{question}
Helpful Answer:
"""
par  = {'grade':'IX', 'subject':'ভ ৌতবিজ্ঞান ও পবিবিশ পবিচয়', 'topic':'ক্ষমতা' }

qa_prompt = PromptTemplate(
    template=qa_template,
    input_variables=['context', 'question'],
    partial_variables = par
)

# define memory
memory = ConversationBufferWindowMemory(
    llm=llm_gemini,
    memory_key="chat_history",
    input_key="question",
    k=5
)


qa_chain_2 = RetrievalQA.from_chain_type(
    llm=llm_gemini,
    chain_type="stuff",
    retriever=docsearch.as_retriever(search_kwargs={"k": 15}),
    chain_type_kwargs={'prompt':qa_prompt, 'memory':memory},
    return_source_documents=True
)

In [ ]:
# Define a function to interact with Mini
def ask_question(question):
    # Retrieve relevant context from Pinecone and generate a response

    response = qa_chain_2.invoke(question)
    return display(Markdown(response['result']))

In [ ]:
ask_question(question="ক্ষমতা কাকে বলে?")

ক্ষমতা হলো কাজ করার হার।

In [ ]:
ask_question(question="Hi")

হ্যালো।

In [ ]:
ask_question(question="Who are you?")

আমি মিনি, একটি কৃত্রিম বুদ্ধিমত্তা-চালিত ভার্চুয়াল সহকারী, যা কে-১২ শিক্ষার্থীদের তাদের পড়াশোনায় সাহায্য করার জন্য ডিজাইন করা হয়েছে।

In [ ]:
ask_question(question="what is your name?")

আমি মিনি, তোমার কৃত্রিম বুদ্ধিমত্তা ভিত্তিক ভার্চুয়াল সহকারী।

In [ ]:
ask_question(question="")

ক্ষমতা হলো কাজ করার হার। এটি কাজের পরিমাণকে সময় দ্বারা ভাগ করে পাওয়া যায়।

In [ ]:
ask_question(question="what are the knowledge you have?")

ক্ষমতা হলো কাজ করার হার। এটি কাজকে সময় দ্বারা ভাগ করে নির্ণয় করা হয়। ক্ষমতার একক হলো ওয়াট (W)।

In [ ]:
ask_question(question="Do you Know me?")

দুঃখিত, আমার কাছে তথ্য নেই।

### **Method 3:**

In [ ]:
from langchain.prompts import ChatPromptTemplate, HumanMessagePromptTemplate, SystemMessagePromptTemplate


In [ ]:
llm_t5 = HuggingFaceHub(repo_id="google/flan-t5-base", huggingfacehub_api_token=HUGGINGFACEHUB_API_TOKEN,
                                   model_kwargs={"temperature":0.4, "max_length":512})

In [ ]:
# System message to initialize the conversation
system_message = SystemMessagePromptTemplate.from_template(
    "You are an AI chatbot that helps K-12 students with their studies."
)

# Define the human messages for context
human_messages = [
    HumanMessagePromptTemplate.from_template("Response based on select Language: {language}"),
    HumanMessagePromptTemplate.from_template("They are studying in class {class}"),
    HumanMessagePromptTemplate.from_template("They select subject, i.e. {subject}"),
    HumanMessagePromptTemplate.from_template("Their topic is {topic}"),
    HumanMessagePromptTemplate.from_template("Ask Question: {query}")
]

# Create the prompt template from system and human messages
prompt_template = ChatPromptTemplate.from_messages([system_message] + human_messages)


prompt_template

ChatPromptTemplate(input_variables=['class', 'language', 'query', 'subject', 'topic'], messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are an AI chatbot that helps K-12 students with their studies.')), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['language'], template='Response based on select Language: {language}')), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['class'], template='They are studying in class {class}')), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['subject'], template='They select subject, i.e. {subject}')), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['topic'], template='Their topic is {topic}')), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['query'], template='Ask Question: {query}'))])

In [ ]:
conversation_chain = LLMChain(
    llm=llm_t5,
    prompt=prompt_template
)

In [ ]:
retriever=docsearch.as_retriever(search_kwargs={"k": 15})

In [ ]:
qa_chains = RetrievalQA.from_chain_type(
    llm=llm_t5,
    chain_type="stuff",
    retriever=retriever,
    return_source_documents=True
)

In [ ]:
# A sequential chain to collect information and then generate responses
chatbot_chain = SequentialChain(
    chains=[conversation_chain, qa_chains],
    input_variables=["language", "class", "subject", "topic", "query"],
    output_variables=["result"]
)

In [ ]:
inputs = {
    "language": "English",
    "class": "8",
    "subject": "Science",
    "topic": "Newton's Laws",
    "query": "Who are You?"
}



In [ ]:
chatbot_chain.invoke(inputs)